In [12]:
import os
#import sys
#import math
#import struct
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#from sklearn.decomposition import PCA
#from sklearn.preprocessing import RobustScaler
#from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split
#from sklearn.feature_selection import SelectFromModel
#from sklearn.feature_selection import VarianceThreshold
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

TEST = True
DATA_DIR = "data/"
dir = "data/data_Q1_2016/"
for file in os.listdir(DATA_DIR):
    print(file)

#yr = '2016'
hd = pd.DataFrame()
for file in os.listdir(dir): # loading 78 indivual data files
    temp = pd.read_csv(dir+file, header=0,nrows=800) # my laptop is struggling to handle more rows
    hd = hd.append(temp)
print("Shape of hd data: ", np.shape(hd))
print("There are %d unique drives. " % hd['serial_number'].value_counts().count())
print("There are %d unique models. " % hd['model'].value_counts().count())
print("There are %d unique dates. " % hd['date'].value_counts().count())
print("There are %d failures." % hd['failure'].sum())

__MACOSX
data_Q1_2016.zip
data_Q1_2016
Shape of hd data:  (72800, 96)
There are 1157 unique drives. 
There are 30 unique models. 
There are 91 unique dates. 
There are 5 failures.


In [2]:
def tb_capacity(x):
    """
    1 gig is np.power(2, 30) bytes, but maybe it is 10^9 bytes. Who you asking? Whatever.
    """
    tb = np.power(10, 12)
    #if not math.isnan(x):
    return x/tb

hd.capacity_bytes = hd.capacity_bytes.map(tb_capacity)
hd.rename(columns={'capacity_bytes': 'capacity_tb'}, inplace=True)
print(hd.groupby('capacity_tb').size())

capacity_tb
-1.000000e-12         2
 8.002636e-02       363
 1.374390e-01         1
 1.600419e-01      2376
 2.500000e-01        91
 2.500594e-01       942
 3.200729e-01      1187
 5.001079e-01     10524
 1.000205e+00       650
 1.500302e+00      1181
 2.000399e+00     53640
 3.000593e+00     81170
 4.000787e+00    547560
 5.000981e+00       479
 6.001175e+00     27554
 8.001563e+00       280
dtype: int64


In [3]:
#tmp = pd.to_datetime(hd.loc[:, 'date'])
#tmp2 = tmp.dt.dayofyear
#hd.rename(columns={'date': 'day_of_year'}, inplace=True)
##print(hd.loc[:, 'date'])
#print(tmp)


In [4]:
hd = pd.DataFrame.copy(hd[hd['model'] == 'ST4000DM000']) # select only the most common seagate model
print(np.shape(hd))
# Some columns have just a few NaNs, while others are all NaNs
# drop columns of all nan
hd_clean = hd.dropna(axis=1, how='all')
# but lets avoid losing columns that potentially have lots of information
# in particular if the fraction of nans is greater than nan_frac, then drop
nan_frac = .3
for ikey in hd_clean.keys():
    tmp = hd_clean[ikey].isnull().sum()
    print(ikey, " has a NaN fraction of:\t\t\t", tmp/len(hd_clean))
    if tmp/len(hd_clean) >= nan_frac:
        hdcl = hd_clean.drop(ikey, 1)

# put zeros in the NaN locations we kept
hd_clean = hd_clean.fillna(0)

for ikey in hd_clean.keys():
    tmp = hd_clean[ikey].isnull().sum()
    if tmp > 0:
        print("Warning, there shouldn't be any NaNs but there are in %s" % ikey)

y = hd_clean.failure
print(np.shape(hd_clean))
print(hd_clean.columns)

(388331, 96)
capacity_tb  has a NaN fraction of:			 0.0
date  has a NaN fraction of:			 0.0
failure  has a NaN fraction of:			 0.0
model  has a NaN fraction of:			 0.0
serial_number  has a NaN fraction of:			 0.0
smart_10_normalized  has a NaN fraction of:			 7.72536830693e-06
smart_10_raw  has a NaN fraction of:			 7.72536830693e-06
smart_12_normalized  has a NaN fraction of:			 7.72536830693e-06
smart_12_raw  has a NaN fraction of:			 7.72536830693e-06
smart_183_normalized  has a NaN fraction of:			 7.72536830693e-06
smart_183_raw  has a NaN fraction of:			 7.72536830693e-06
smart_184_normalized  has a NaN fraction of:			 7.72536830693e-06
smart_184_raw  has a NaN fraction of:			 7.72536830693e-06
smart_187_normalized  has a NaN fraction of:			 7.72536830693e-06
smart_187_raw  has a NaN fraction of:			 7.72536830693e-06
smart_188_normalized  has a NaN fraction of:			 7.72536830693e-06
smart_188_raw  has a NaN fraction of:			 7.72536830693e-06
smart_189_normalized  has a NaN fraction 

In [5]:
def clip_by_column_txt(df, col_txt):
    for ikey in df.keys():
        if col_txt not in ikey and ikey != 'serial_number' and ikey != 'failure' and ikey != 'date':
            del df[ikey]
        
#serial_number = hd_clean['serial_number']
#failure = hd_clean['failure']
clip_by_column_txt(hd_clean,"raw") # selected columns and those that have "raw"

print(np.shape(hd_clean))
#print(np.shape(serial_number))
#print(np.shape(failure))
print(hd_clean.columns)

(388331, 27)
Index(['date', 'failure', 'serial_number', 'smart_10_raw', 'smart_12_raw',
       'smart_183_raw', 'smart_184_raw', 'smart_187_raw', 'smart_188_raw',
       'smart_189_raw', 'smart_190_raw', 'smart_191_raw', 'smart_192_raw',
       'smart_193_raw', 'smart_194_raw', 'smart_197_raw', 'smart_198_raw',
       'smart_199_raw', 'smart_1_raw', 'smart_240_raw', 'smart_241_raw',
       'smart_242_raw', 'smart_3_raw', 'smart_4_raw', 'smart_5_raw',
       'smart_7_raw', 'smart_9_raw'],
      dtype='object')


In [6]:
# aggregations = {
#     'smart_9_raw': { # smart 9 is the disk uptime
#         'runtime_max': 'max',  
#         'runtime_min': 'min',
#         'uptime': lambda x: max(x) - min(x)  
#     },
#     'failure': {
#      'failure': 'sum'
#     }  
# }
# survival = hd_clean.groupby('serial_number').agg(aggregations).reset_index()
# survival.columns = survival.columns.droplevel()
# survival = survival.loc[survival['failure'] <=1 ] # if the sum of failures is gt 1 then then drive's data is wonky
# survival['runtime_max'] = survival['runtime_max']/8760.0 # hrs to yrs
# survival['runtime_min'] = survival['runtime_min']/8760.0
#print(survival['failure'].value_counts())

In [7]:
#print(serial_number.unique().counts())
#print(hd['serial_number'].value_counts().count())
#print(serial_number.value_counts().count())
#print(np.shape(serial_number.unique()))
#unique_serial_numbers = serial_number.unique()

In [8]:
print(hd_clean['serial_number'].value_counts().count())

6001


## Need to group all observations that relate to a single HD

In [9]:
serial_grouped = hd_clean.groupby(['serial_number'])

In [10]:
serial_grouped = list(serial_grouped)

In [11]:
serial_data_dir = 'serial_data/'
for i in range((len(serial_grouped))):
    fname = serial_data_dir + serial_grouped[i][0] + '.csv'
    indiv_serial_data = serial_grouped[i][1]
    f=open(fname,'w')
    f.write("#"+ str(np.shape(indiv_serial_data)) + "\n#")
    f.close()
    del  indiv_serial_data['serial_number']    
    indiv_serial_data = indiv_serial_data.sort_values(by=['smart_9_raw']) 
    indiv_serial_data.to_csv(path_or_buf=fname, sep=',', header=True, index=False, mode='a', encoding='utf-8') # change mode to 'a' to append)
    if i>= 1000:
            break